In [1]:
import os
import time

import tools
import benchmarks as bm
from poseEvaluation import PoseErrorEvaluator
from evo.core import metrics

[DEBUG][2024-04-30 23:42:29,546][log.configure_logging():112]
System info:
Python 3.8.17
Linux-6.5.0-28-generic-x86_64-with-glibc2.17
slam@slam-TensorBook



In [ ]:
benchmark_list = ["SenseTime","LivingRoom", "Hall", "Lab", "Lab2"
                  "Apartment","AbandonedFactory","FireStationGarage",
                  "FireStationKitchen","FireStationOffice"]
root_dir = "../DeepSEE_SLAM"
trials = 5
delta = 10
max_diff = 0.05
max_null_length = 10

os.chdir(root_dir)
root_dir = "."

In [3]:
poseErrorEvaluator = PoseErrorEvaluator(root_dir, delta, max_diff, max_null_length)

In [4]:
tools.delete_files_in_folder("./logs")
tools.delete_files_in_folder("./logs/distribution")
tools.delete_files_in_folder("./data", True)
tools.delete_files_in_folder("./figures")

Skipping: ./logs/distribution (It is not a file)
All files in ./logs deleted successfully.
All files in ./logs/distribution deleted successfully.
Successfully deleted ./data/dist_LivingRoom_A2_4
Successfully deleted ./data/dist_LivingRoom_A1_2
Successfully deleted ./data/dist_LivingRoom_A3_2
Successfully deleted ./data/dist_LivingRoom_A4_4
Successfully deleted ./data/dist_LivingRoom_A4_2
Successfully deleted ./data/dist_LivingRoom_A1_1
Successfully deleted ./data/dist_LivingRoom_A3_3
Successfully deleted ./data/dist_LivingRoom_A2_1
Successfully deleted ./data/dist_LivingRoom_A3_1
Successfully deleted ./data/raw
Successfully deleted ./data/dist_LivingRoom_A0_2
Successfully deleted ./data/dist_LivingRoom_A6_3
Successfully deleted ./data/dist_LivingRoom_A4_3
Successfully deleted ./data/dist_LivingRoom_A5_4
Successfully deleted ./data/dist_LivingRoom_A3_4
Successfully deleted ./data/dist_LivingRoom_A6_1
Successfully deleted ./data/dist_LivingRoom_A6_0
Successfully deleted ./data/dist_Livin

In [5]:
for i, benchmark in enumerate(benchmark_list):
    benchmarkClass = bm.benchmark_factory[benchmark]
    benchmarkObject = benchmarkClass(root_dir)
    benchmarkObject.generate_script()
    scriptList = benchmarkObject.get_script()
    
    for j, scriptDict in enumerate(scriptList):    
        time.sleep(1)
        benchmark = scriptDict["benchmark"]
        trajectory = scriptDict["trajectory"]
        benchmarkObject.copy_ground_truth_traj(scriptDict["trajectory"])
        
        for trial in range(0, trials):
            
            folder_path = "./logs/distribution"
            if not os.path.exists(folder_path):
                # Create the folder
                os.makedirs(folder_path)
                print("Folder created.")
            else:
                print("Folder already exists.")
            
            consoleCommand = "ulimit -s 64000 && " + scriptDict['script']
            print(consoleCommand)
            try:
                tools.runCommand(consoleCommand, consoleOutputEncoding="utf-8", timeout=360)
                #os.system(scriptDict['script'])
            except:
                print("="*50)
                print("{}-{}-{} Timeout. Skipped".format(benchmark, trajectory, trial))
                print("="*50)
                continue
            time.sleep(1)
            
            benchmarkObject.copy_raw_SLAM_data(benchmark,trajectory,trial)
            time.sleep(1)
            
            df, df_traj, initial_rows = benchmarkObject.load_n_save_estimated_traj()
            time.sleep(1)
            
            poseErrorEvaluator.load_trajectory()
            traj_est, traj_ref = poseErrorEvaluator.get_traj_w_gt()
            
            rpe_metric = poseErrorEvaluator.calculate_RE(pose_relation=metrics.PoseRelation.point_distance)
            #rpe_metric = poseErrorEvaluator.calculate_RE(pose_relation=metrics.PoseRelation.point_distance_error_ratio)
            
            poseErrorEvaluator.merge_feature_with_label()
            merged_df = poseErrorEvaluator.get_feature_w_label()
            error_df = poseErrorEvaluator.get_error_df()
            
            poseErrorEvaluator.plot_aligned_trajectory(traj_est, traj_ref, benchmark, trajectory, trial)
            #poseErrorEvaluator.plot_trajectory(traj_est, traj_ref, benchmark, trajectory, trial)
            
            poseErrorEvaluator.plot_error(error_df, delta, benchmark, trajectory, trial, plot_error_min = -0.01, plot_error_max=0.1)
            
            poseErrorEvaluator.plot_trajectory_with_error(rpe_metric, traj_ref, traj_est,
                                                          benchmark, trajectory, trial)
            
            
            #df_data = poseErrorEvaluator.attach_label_2_features(delta, error, df)
            #time.sleep(1)

            merged_df.to_csv("./data/data_{}_{}_{}.csv".format(benchmark,trajectory,trial), sep=',', index=False, header=True)
            dist_target = "./data/dist_{}_{}_{}".format(benchmark,trajectory,trial)
            dist_source = "./logs/distribution"
            tools.delete_folder(dist_target)
            tools.copy_folder(dist_source, dist_target)
            
            time.sleep(2)
            
            tools.copy_file_and_rename("./PythonAPI/RuntimeLogs/log.csv", \
                                       "./PythonAPI/RuntimeLogs/", 
                                       "{}_{}_{}.csv".format(benchmark,trajectory,trial))
            time.sleep(1)
            os.remove("./PythonAPI/RuntimeLogs/log.csv")
            
            time.sleep(1)
            
            print("="*50)
            print("="*50)

Folder already exists.
ulimit -s 64000 && ./Examples/Monocular/mono_tum_vi ./Vocabulary/ORBvoc.txt ./datasets/LivingRoom/config.yaml ./datasets/LivingRoom/A0/mav0/cam0/data ./datasets/LivingRoom/A0/A0.txt
File copied successfully and renamed to 'raw_LivingRoom_A0_0.csv' in './data/raw'.
[DEBUG][2024-04-30 23:44:41,299][file_interface.read_tum_trajectory_file():117]
Loaded 1078 stamps and poses from: ./logs/trajectory.txt
[DEBUG][2024-04-30 23:44:41,376][file_interface.read_tum_trajectory_file():117]
Loaded 24578 stamps and poses from: ./logs/ground_truth.txt
Potential subtrajectories for alignment: [[0, 200], [200, 400], [400, 424], [424, 472], [473, 673], [673, 873], [873, 1073], [1073, 1077]]
Original estimated subtrajectory length: 200
[DEBUG][2024-04-30 23:44:41,469][file_interface.read_tum_trajectory_file():117]
Loaded 24578 stamps and poses from: ./logs/ground_truth.txt
[DEBUG][2024-04-30 23:44:41,478][sync.associate_trajectories():110]
Found 200 of max. 200 possible matching tim

Loaded trajectory (1076 poses)  with ground truth 1076 poses
[DEBUG][2024-04-30 23:44:42,513][metrics.id_pairs_from_delta():469]
Found 1066 pairs with delta 10 (frames) among 1076 poses using all pairs.
[DEBUG][2024-04-30 23:44:42,518][metrics.process_data():313]
Compared 0 relative pose pairs, delta = 10 (frames) with all pairs.
[DEBUG][2024-04-30 23:44:42,519][metrics.process_data():319]
Calculating RPE for point distance pose relation...
Loaded trajectory with ground truth
aligned length: 538
[DEBUG][2024-04-30 23:44:43,212][trajectory.align():202]
Aligning using Umeyama's method... (with scale correction)
[DEBUG][2024-04-30 23:44:43,214][trajectory.align():214]
Rotation of alignment:
[[ 9.99999949e-01 -5.44503999e-05 -3.13586849e-04]
 [ 5.43922783e-05  9.99999981e-01 -1.85350073e-04]
 [ 3.13596936e-04  1.85333007e-04  9.99999934e-01]]
Translation of alignment:
[ 0.00098453  0.00080358 -0.0002394 ]
[DEBUG][2024-04-30 23:44:43,215][trajectory.align():216]
Scale correction: 1.00041186

[DEBUG][2024-04-30 23:46:00,559][trajectory.align():214]
Rotation of alignment:
[[ 9.99998286e-01  4.95735928e-04  1.78381763e-03]
 [-4.95992352e-04  9.99999867e-01  1.43310499e-04]
 [-1.78374634e-03 -1.44195013e-04  9.99998399e-01]]
Translation of alignment:
[-0.00255271 -0.00018622 -0.00015769]
[DEBUG][2024-04-30 23:46:00,559][trajectory.align():216]
Scale correction: 0.9995496012670321
The folder ./data/dist_LivingRoom_A0_1 does not exist.
File copied successfully and renamed to 'LivingRoom_A0_1.csv' in './PythonAPI/RuntimeLogs/'.
Folder created.
ulimit -s 64000 && ./Examples/Monocular/mono_tum_vi ./Vocabulary/ORBvoc.txt ./datasets/LivingRoom/config.yaml ./datasets/LivingRoom/A0/mav0/cam0/data ./datasets/LivingRoom/A0/A0.txt



(mono_tum_vi:6962): Gtk-CRITICAL **: 23:47:14.309: IA__gtk_window_get_size: assertion 'GTK_IS_WINDOW (window)' failed

(mono_tum_vi:6962): Gtk-CRITICAL **: 23:47:14.309: IA__gtk_widget_get_screen: assertion 'GTK_IS_WIDGET (widget)' failed


File copied successfully and renamed to 'raw_LivingRoom_A0_2.csv' in './data/raw'.
[DEBUG][2024-04-30 23:47:17,359][file_interface.read_tum_trajectory_file():117]
Loaded 1087 stamps and poses from: ./logs/trajectory.txt
[DEBUG][2024-04-30 23:47:17,484][file_interface.read_tum_trajectory_file():117]
Loaded 24578 stamps and poses from: ./logs/ground_truth.txt
Potential subtrajectories for alignment: [[0, 200], [200, 400], [400, 600], [600, 800], [800, 1000], [1000, 1086]]
Original estimated subtrajectory length: 200
[DEBUG][2024-04-30 23:47:17,569][file_interface.read_tum_trajectory_file():117]
Loaded 24578 stamps and poses from: ./logs/ground_truth.txt
[DEBUG][2024-04-30 23:47:17,577][sync.associate_trajectories():110]
Found 200 of max. 200 possible matching timestamps between...
	first trajectory
and:	second trajectory
..with max. time diff.: 0.05 (s) and time offset: 0.0 (s).
[DEBUG][2024-04-30 23:47:17,578][trajectory.align():202]
Aligning using Umeyama's method... (with scale correc

...


(mono_tum_vi:84358): Gtk-CRITICAL **: 04:00:04.343: IA__gtk_widget_get_toplevel: assertion 'GTK_IS_WIDGET (widget)' failed


File copied successfully and renamed to 'raw_Lab2_A7_2.csv' in './data/raw'.
[DEBUG][2024-05-01 04:00:07,397][file_interface.read_tum_trajectory_file():117]
Loaded 1089 stamps and poses from: ./logs/trajectory.txt
[DEBUG][2024-05-01 04:00:07,488][file_interface.read_tum_trajectory_file():117]
Loaded 30159 stamps and poses from: ./logs/ground_truth.txt
Potential subtrajectories for alignment: [[0, 200], [200, 400], [400, 600], [600, 800], [800, 1000], [1000, 1088]]
Original estimated subtrajectory length: 200
[DEBUG][2024-05-01 04:00:07,593][file_interface.read_tum_trajectory_file():117]
Loaded 30159 stamps and poses from: ./logs/ground_truth.txt
[DEBUG][2024-05-01 04:00:07,602][sync.associate_trajectories():110]
Found 200 of max. 200 possible matching timestamps between...
	first trajectory
and:	second trajectory
..with max. time diff.: 0.05 (s) and time offset: 0.0 (s).
[DEBUG][2024-05-01 04:00:07,603][trajectory.align():202]
Aligning using Umeyama's method... (with scale correction)


[DEBUG][2024-05-01 04:01:39,088][sync.associate_trajectories():110]
Found 200 of max. 200 possible matching timestamps between...
	first trajectory
and:	second trajectory
..with max. time diff.: 0.05 (s) and time offset: 0.0 (s).
[DEBUG][2024-05-01 04:01:39,089][trajectory.align():202]
Aligning using Umeyama's method... (with scale correction)
[DEBUG][2024-05-01 04:01:39,091][trajectory.align():214]
Rotation of alignment:
[[-0.98228584  0.18292303 -0.04066558]
 [ 0.18083564  0.8684569  -0.46160706]
 [-0.04912225 -0.46078386 -0.88615193]]
Translation of alignment:
[-0.41697632  0.34488626  0.89720567]
[DEBUG][2024-05-01 04:01:39,091][trajectory.align():216]
Scale correction: 2.1359321655309684
Original estimated subtrajectory length: 200
[DEBUG][2024-05-01 04:01:39,189][file_interface.read_tum_trajectory_file():117]
Loaded 30159 stamps and poses from: ./logs/ground_truth.txt
[DEBUG][2024-05-01 04:01:39,198][sync.associate_trajectories():110]
Found 200 of max. 200 possible matching times

Unable to open file: ./logs/log.csv


File copied successfully and renamed to 'raw_Lab2_A7_4.csv' in './data/raw'.
[DEBUG][2024-05-01 04:03:09,666][file_interface.read_tum_trajectory_file():117]
Loaded 1115 stamps and poses from: ./logs/trajectory.txt
[DEBUG][2024-05-01 04:03:09,766][file_interface.read_tum_trajectory_file():117]
Loaded 30159 stamps and poses from: ./logs/ground_truth.txt
Potential subtrajectories for alignment: [[0, 200], [200, 400], [400, 600], [600, 800], [800, 1000], [1000, 1114]]
Original estimated subtrajectory length: 200
[DEBUG][2024-05-01 04:03:10,430][file_interface.read_tum_trajectory_file():117]
Loaded 30159 stamps and poses from: ./logs/ground_truth.txt
[DEBUG][2024-05-01 04:03:10,439][sync.associate_trajectories():110]
Found 200 of max. 200 possible matching timestamps between...
	first trajectory
and:	second trajectory
..with max. time diff.: 0.05 (s) and time offset: 0.0 (s).
[DEBUG][2024-05-01 04:03:10,440][trajectory.align():202]
Aligning using Umeyama's method... (with scale correction)
